In [ ]:
import pandas as pd
from tqdm.auto import tqdm
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

In [ ]:
from glob import glob

In [ ]:
dataset = load_dataset('csv', data_files={"train": "/home/karysoares/Documents/book-reviews/data/sentiment_train.csv", "test": "/home/karysoares/Documents/book-reviews/data/sentiment_test.csv"})

In [ ]:
labels = [1, 2, 3, 4, 5]

id2label = {n: i for n, i in enumerate(labels)}
label2id = {i: n for n, i in enumerate(labels)}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

def preprocess_function(examples):
    inputs = tokenizer(examples["text"], truncation=True, padding=True, return_tensors="pt")
    inputs["label"] = [label2id[label] for label in examples["label"]]
    return inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, batch_size=100)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
accuracy = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "nlptown/bert-base-multilingual-uncased-sentiment", num_labels=len(labels), id2label=id2label, label2id=label2id
)

In [ ]:
training_args = TrainingArguments(
    output_dir="/home/karysoares/Documents/book-reviews/notebooks/genre_classification",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()